In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [2]:
                                                                ##########################
                                                                #### IMPORT LIBRARIES ####
                                                                ##########################

from colorama import Fore, Back, Style
print(Fore.RED,'/!\ Importation des Librairies /!\ ',Style.RESET_ALL)
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

import pandas_datareader as web
import time
import datetime as dt
import os

import talib
from sklearn.preprocessing import MinMaxScaler
import yaml
from keras.models import model_from_yaml
from sklearn.metrics import  precision_score
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
yaml.warnings({'YAMLLoadWarning': False})
warnings.filterwarnings("ignore")
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import sys
import codecs
%matplotlib inline
plt.style.use('seaborn')
print(Fore.GREEN,'  Librairies importées avec succès',Style.RESET_ALL)

 /!\ Importation des Librairies /!\  
Using TensorFlow backend.
   Librairies importées avec succès 


In [3]:
                                                                ###############################
                                                                #### INITIALISATION LISTES ####
                                                                ###############################

LIST_DIR = os.listdir('DL_2019-12-06/.')
NEW_LIST = []
for deepl in range(0,len(str(LIST_DIR).split(','))):
    if str(LIST_DIR).split(',')[deepl].split(',')[0].split('.')[1].split("'")[0]=='yaml':
        NEW_LIST.append(str(LIST_DIR).split(',')[deepl].split(',')[0].split('.')[0].split("'")[1].split('_')[1])
NEW_LIST = sorted(set(NEW_LIST))

GREEN = []
RED = []
GREEN_CLOSE = []
RED_CLOSE=[]
TICKER = []
DATE = []
PRICE = []
OPEN_S = []
CONFUP = []
CONFDWN = []
PERUP = []
PERDOWN = []
DAY = []
ERRORDATE = []
ERRORLONG = []

WINNERS = []
LOOSERS = []

POS_DURATION = []
TRADES = []
EQ_CURX = []
EQ_CURY = []
MAXDD = []

ERROR_3 = []

In [4]:
                                                                ##################################
                                                                #### INITIALISATION VARIABLES ####
                                                                ##################################

orig_stdout = sys.stdout
LeChemin = 'DL_2019-12-06/'
start = '2015-12-06'
split = '2020-01-21'
end = '2020-01-23'

suffixe = 'minitest'

long = len(NEW_LIST)

filtre_up = 93
filtre_down = 97
filtre_up_close = 80
filtre_down_close = 80

nb_win = 0
nb_los = 0
mini_pnl = 0
nbS = 0
nbL = 0

pos_long = 0
pos_short = 0

pactol = 1000000
mise = 30000
price_buy = 0
price_sell = 0
x = 0

average_duration = pd.Timedelta('0 days 00:00:00')


In [5]:
                                                                ##################################
                                                                #### DEFINITION DES FONCTIONS ####
                                                                ##################################

def scrap_data(ticker,start,end):
    df = web.DataReader(ticker,'yahoo',start,end)
    df = df.drop(['Close'],axis=1)
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis = 1)    
    return(df)

def load_boot(df,model):
    seed = 770
    np.random.seed(seed)
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    return(df)


def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown #(aroonup-aroondown)/abs((aroonup-aroondown))
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        rsi70_list.append(70 - rsi[i])
        rsi30_list.append(rsi[i] - 30)
        #except:
         #   rsi70_list.append(0)
          #  rs30_list.append(0)
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    
    df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
    df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]):
        if df.iloc[i]['%Futur'] > 0.5 :#or df.iloc[i]['%Futur2'] > 0.1:
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur'] < -0.5: #or df.iloc[i]['%Futur2'] < -0.1:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)

    df['target_up'] = up  # target_up # abs(np.array(valley))#target_up
    df['target_down'] = down # target_down # abs(np.array(peak))#target_down
    df = df[['High','Low','Open','Volume','Close','Varop_Spy','Varhl_spy','RSI','70 - RSI','RSI - 30',\
             'BBD_Delta_Up','delta5_8','delta8_10','delta10_12','delta12_15','delta15_30','delta30_35',\
             'delta35_40','delta40_45','delta45_50','Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper',\
             'lower','var_bollup_kchigh','var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon','Close.S',\
             'Close.S2','%Futur','%Futur2','target_up','target_down']]
    #df = df.dropna()
    return(df)

def deep_learning(df):
    seed = 770
    np.random.seed(seed)
    X = df.copy()
    #####################X = X.iloc[:,1:-4]
    X = X.iloc[:,:-4]
    X.astype(np.float64)
    y_up = df.iloc[:,-2].values
    y_down = df.iloc[:,-1].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 

    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:].strftime('%Y-%m-%d')
    resultats.index= df.index[bloc1:]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close.S']
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    
    return(resultats,precision_up,precision_down,scaler)

def genere_base():
    print(Fore.YELLOW,'ON GENERE LaBase',Style.RESET_ALL)
    tmps55=time.time()
    LaBase = pd.DataFrame()
    df = pd.DataFrame()
    BADTICKER = ['AMCR']
    for loop,ticker in enumerate(NEW_LIST):
        try:
            df = web.DataReader(ticker,'yahoo',start,end)
            df['Date'] = df.index
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        except:
            BADTICKER.append(ticker)
            continue
        df['Symbol'] = ticker
        df = df[['Symbol','High','Low','Open','Close','Volume','Adj Close']]
        LaBase = pd.concat((LaBase,df),ignore_index=False)
    tmps22=round(time.time()-tmps55,2)
    print(Fore.BLUE,"Time for completing LaBase Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

    print('Il y a ',len(BADTICKER),' mauvais tickers')
    print('Les voici')
    print(BADTICKER)

    print('Base générée : ',LaBase.shape)
    print('Il y a ',LaBase[LaBase.duplicated()].shape,' Duplicats')
    print('Hors AMCR, voici les duplicats') 
    print(LaBase[(LaBase.duplicated()==True)&(LaBase['Symbol'] != 'AMCR')])
    try:
        os.remove('LaBase_17.csv')
    except:
        pass
    LaBase.to_csv('LaBase_17.csv') 
    return(LaBase)

def charge_base():
    LaBase = pd.read_csv('LaBase_adj.csv')
    try:
        LaBase = LaBase.drop(['Unnamed: 0'],axis=1)
    except :
        pass
    LaBase.index = pd.to_datetime(LaBase['Date'],format='%Y-%m-%d')
    try:
        LaBase = LaBase.drop(['Date'],axis=1)
        #LaBase = LaBase.drop(['Close'],axis=1)
        LaBase['Close'] = LaBase['Adj Close']
    except:
        pass
    try:
        LaBase = LaBase.drop(['Adj Close'],axis=1)
    except:
        pass
    LaBase = LaBase.sort_index()
    return(LaBase)
def charge_base_prep():
    BasePrep = pd.read_csv('BasePrep.csv')
    try:
        BasePrep = BasePrep.drop(['Unnamed: 0'],axis=1)
    except :
        pass
    BasePrep.index = pd.to_datetime(BasePrep['Date'],format='%Y-%m-%d')
    try:
        BasePrep = BasePrep.drop(['Date'],axis=1)
    except:
        pass
    try:
        BasePrep['Close'] = BasePrep['Adj Close']
    except:
        pass
    try:
        BasePrep = BasePrep.drop(['Adj Close'],axis=1)
    except:
        pass
    BasePrep = BasePrep[['Symbol','High','Low','Open','Volume','Close',\
                            'Varop_Spy','Varhl_spy','RSI','70 - RSI','RSI - 30',\
                            'BBD_Delta_Up','delta5_8','delta8_10','delta10_12',\
                            'delta12_15','delta15_30','delta30_35','delta35_40',\
                            'delta40_45','delta45_50','Stoc_Slowk','Stoc_Slowd',\
                            'KC_High','KC_Low','upper','lower','var_bollup_kchigh',\
                            'var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon',\
                            'Close.S','Close.S2','%Futur','%Futur2','target_up','target_down']]
    BasePrep = BasePrep.sort_index()
    return(BasePrep)

def trace_perf(Signals):
    plt.figure(figsize=(10,4))
    plt.plot(Signals.index,Signals['Conf Up'],label='Confiance Up')
    plt.legend()
    plt.figure(figsize=(10,4))
    plt.plot(Signals.index,Signals['Conf Dwn'],label='Confiance Down')
    plt.legend()
    plt.figure(figsize=(10,4))
    plt.plot(Signals.index,Signals['% up'],label='Precision Up')
    plt.legend()
    plt.figure(figsize=(10,4))
    plt.plot(Signals.index,Signals['% down'],label='Precision Down')
    plt.legend()
    return

In [6]:
                                                                    #####################
                                                                    ###### CORPUS #######
                                                                    #####################

################################
#### GENERATION DES SIGNAUX ####
################################

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5780(c)')
print(Fore.BLUE,'°Go!em 73°')
print('ULTIMATE BACKTEST v0.5',Style.RESET_ALL)

# CHARGEMENT DES BASES
print('\n\nChargement des Bases')
LaBase = charge_base()
BasePrep = charge_base_prep()

# DEFINITION BLOC1
delta = round(LaBase.shape[0])
bloc1 =  round(0.75 * LaBase[LaBase['Symbol']=='A'].shape[0]) #round(delta*0.80)
bloc2 = delta - bloc1
print('\nBloc1 = ',bloc1)

# RESERVATION DE L'OPEN.S DANS BASEPREP
BasePrep['Open.S'] = BasePrep['Open'].shift(-1)

# RESTRICTION DE LA BASE ET DE BASEPREP DEPUIS L'ORIGINE DE TOUT JUSQU'A END
#LaBase = LaBase[(LaBase.index>=start)&(LaBase.index<end)]


# LISTE DES JOURS DE BACKTEST
DAY = sorted(set(BasePrep[BasePrep.index>=pd.to_datetime(split,format='%Y-%m-%d')].index))


# BOUCLE JOUR PAR JOUR

print(Fore.GREEN,'\nGENERATION DES SIGNAUX',Style.RESET_ALL)

tmps55=time.time()

print(Fore.RED,'/!\ ATTENTION /!\ ',Style.RESET_ALL,'La dernière date est',DAY[-1])

print('La dernière date',end,' a été enlevée de la base')


print('Double vérification de la dernière date de BasePrep :',str(BasePrep.index[-1])[:10])

for day in DAY:
    BasePrepDay = BasePrep[BasePrep.index<=day]
    print('\nJour testé : ',str(day)[0:10])

    for loop,ticker in enumerate(NEW_LIST):
    
        if loop == 1:
                print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
        if loop == round(1 * long / 9):
            print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
        if loop == round(2 * long / 9):
            print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
        if loop == round(3 * long / 9):
            print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
        if loop == round(4 * long / 9):
            print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
        if loop == round(5 * long / 9):
            print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
        if loop == round(6 * long / 9):
            print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
        if loop == round(7 * long / 9):
            print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
        if loop == round(8 * long / 9):
            print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',Style.RESET_ALL,end='',flush=True)
        
        yamlup = (LeChemin+'Save_'+ticker+'_up.yaml')
        yamldown = (LeChemin+'Save_'+ticker+'_down.yaml')
        modelup = (LeChemin+'Save_'+ticker+'_up.h5')
        modeldown = (LeChemin+'Save_'+ticker+'_down.h5')

        yaml_file_up = open(yamlup, 'r')
        yaml_file_down = open(yamldown, 'r')
        model_yaml_up = yaml_file_up.read()
        model_yaml_down = yaml_file_down.read()
        yaml_file_up.close()
        yaml_file_down.close()
        model_up = model_from_yaml(model_yaml_up)
        model_down = model_from_yaml(model_yaml_down)
        
        # load weights into new model
        model_up.load_weights(modelup)
        model_down.load_weights(modeldown)

        df = BasePrepDay[BasePrepDay['Symbol']==ticker]

        if df.shape[0] < 70:
            ERRORLONG.append(ticker)
            continue
        '''elif str(df.index[-1])[:10] != day :
            ERRORDATE.append(ticker)
            continue'''
        
        df = df.drop(['Symbol'],axis=1)
                 
        df = df[['High','Low','Open','Volume','Close','Varop_Spy',\
                'Varhl_spy','RSI','70 - RSI','RSI - 30','BBD_Delta_Up',\
                'delta5_8','delta8_10','delta10_12','delta12_15','delta15_30',\
                'delta30_35','delta35_40','delta40_45','delta45_50',\
                'Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper',\
                'lower','var_bollup_kchigh','var_bolllow_kclow','Aroon Up',\
                'Aroon Down','Delta Aroon','Close.S','Close.S2','%Futur','%Futur2','target_up','target_down']]

        df.to_csv('BackTest/PrepData/prepdata_'+ticker+suffixe+'.csv')
    
        resultats,precision_up,precision_down,scaler = deep_learning(df)

        # RESTRICTION DE RESULTATS DEPUIS L'ORIGINE DE TOUT JUSQU'A END
        #LaBase = LaBase[(LaBase.index>=start)&(LaBase.index<end)]

        resultats = resultats[(resultats.index>=start)&(resultats.index<end)]
        

        if (precision_up * 100) > 87 and (precision_down * 100) > 87:

            ####################
            ##### SIGNALS #####
            ###################

            price = resultats.iloc[-1]['Actual']

            if resultats.iloc[-1]['Confiance Down'] > filtre_down:
                doob = -1
                waab = 0
                

            elif resultats.iloc[-1]['Confiance Down'] > filtre_down_close and resultats.iloc[-1]['Confiance Down'] < filtre_down :
                waab = -1
                doob = 0
                

            elif resultats.iloc[-1]['Confiance up'] > filtre_up:
                doob = 1
                waab = 0
                

            elif resultats.iloc[-1]['Confiance up'] > filtre_up_close and resultats.iloc[-1]['Confiance up'] < filtre_up:
                waab = 1
                doob = 0
                

            else :
                waab = 0
                doob = 0
                

            if doob == 1 :
                DATE.append(resultats.iloc[-1]['Date'])
                TICKER.append(ticker)
                GREEN.append(1)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[-1]['Open.S'])
                CONFUP.append(resultats.iloc[-1]['Confiance up'])
                CONFDWN.append(resultats.iloc[-1]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue

            if doob == -1 :
                DATE.append(resultats.iloc[-1]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(1)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[-1]['Open.S'])
                CONFUP.append(resultats.iloc[-1]['Confiance up'])
                CONFDWN.append(resultats.iloc[-1]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue
                
            if doob == 0 and waab == -1 :
                DATE.append(resultats.iloc[-1]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[-1]['Open.S'])
                CONFUP.append(resultats.iloc[-1]['Confiance up'])
                CONFDWN.append(resultats.iloc[-1]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue
                
            if doob == 0 and waab == 1 :
                DATE.append(resultats.iloc[-1]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[-1]['Open.S'])
                CONFUP.append(resultats.iloc[-1]['Confiance up'])
                CONFDWN.append(resultats.iloc[-1]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue
                
            if doob == 0 and waab == 0 :
                DATE.append(resultats.iloc[-1]['Date'])
                TICKER.append(ticker)
                GREEN.append(999)
                RED.append(999)
                GREEN_CLOSE.append(999)
                RED_CLOSE.append(999)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[-1]['Open.S'])
                CONFUP.append(resultats.iloc[-1]['Confiance up'])
                CONFDWN.append(resultats.iloc[-1]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue
Signals = pd.DataFrame()
Signals['Date'] = DATE
Signals['Ticker'] = TICKER
Signals['Conf Up'] = CONFUP
Signals['Conf Dwn'] = CONFDWN
Signals['Green'] = GREEN
Signals['Red'] = RED
Signals['Green_Close'] = GREEN_CLOSE
Signals['Red_Close'] = RED_CLOSE
Signals['Price'] = PRICE
Signals['Open.S'] = OPEN_S
Signals['% up'] = PERUP
Signals['% down'] = PERDOWN


tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\n Time for completing Signal Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

Signals.to_csv('Backtest/Signals_'+suffixe+'.csv')



Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5780(c)
 °Go!em 73°
ULTIMATE BACKTEST v0.5 


Chargement des Bases

Bloc1 =  791
 
GENERATION DES SIGNAUX 
 /!\ ATTENTION /!\   La dernière date est 2020-02-18 00:00:00
La dernière date 2020-01-23  a été enlevée de la base
Double vérification de la dernière date de BasePrep : 2020-02-18

Jour testé :  2020-01-21





   |==================  O  ==| 
Jour testé :  2020-01-22
   |==================  O  ==| 
Jour testé :  2020-01-23
   |====  O  ================|

KeyboardInterrupt: 

In [ ]:
 BasePrep.tail()

In [19]:
resultats.tail()

,Date,Move Up,Confiance up,Move Down,Confiance Down,Actual,Actual.S,Open.S,DMP_CP
Date,,,,,,,,,
2020-01-15,2020-01-15,0,0.000054,1,99.997833,50.369999,48.168079,50.709999,-99.999893
2020-01-16,2020-01-16,0,0.001097,1,98.314072,50.740002,49.603882,50.919998,-99.997765
2020-01-17,2020-01-17,0,0.000185,1,99.989655,50.740002,48.791435,50.580002,-99.999634
2020-01-21,2020-01-21,0,0.000195,1,99.995888,49.900002,47.651497,47.660000,-99.999603
2020-01-22,2020-01-22,0,0.017014,1,98.817909,47.840000,46.793610,NaN,-99.965576


In [9]:
###############################
#### SIMULATION DU TRADING ####
###############################

Signals = pd.read_csv('BackTest/Signals_'+suffixe+'.csv')

try:
    Signals = Signals.drop(['Unnamed: 0'],axis=1)
except :
    pass
Signals.index = pd.to_datetime(Signals['Date'],format='%Y-%m-%d')


TICKER = sorted(set(Signals['Ticker']))
#DATE = sorted(set(Signals.index.strftime('%Y-%m-%d')))

trades = pd.DataFrame(columns = ['Date','Ticker','Open Long','Close Long','Open Short','Close Short','Close_position_price','Buy/Sell Price',\
                                  'Quantity','Actual','PNL','Latent PNL','Duration '])

backtest = pd.DataFrame(columns = ['Symb.','Name','Sector','Period','Begin.','End','Pnl $','Pnl %','Gross Profit','Gross Loss','# Winners','# Loosers',\
                          '% Winners','MaxDD $','MaxDD %','Aver. Win $','Aver. Loos $','Aver. Duration','Total Trades','% Long','% Short','Profit Factor'])


print('SIMULATION DU TRADING')

tmps55=time.time()

for ticker in TICKER:

    f = codecs.open('BackTest/BT/backtest de '+str(ticker)+'.txt', 'w','utf-8')
    f.write(u'\ufeff')
    sys.stdout = f

    print('\nTICKER : ',ticker)
    signals = Signals[Signals['Ticker'] == ticker]
    if signals.shape[0] < 1:
        print('Pour le Ticker',ticker,'nous avons un problème de longueur')
        ERROR_3.append(ticker)
        continue
    for i in range(0,len(signals)):
        if pos_long == 0 and pos_short == 0 and signals.iloc[i]['Green'] == 1 :
            pos_long = 1
            nbL+=1
            x = round((mise)/signals.iloc[i]['Open.S'])
            price_buy = round(signals.iloc[i]['Open.S'],2)
            time_enter = signals.iloc[i]['Date']

            trades = trades.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 1,'Close Long' : 0, 'Open Short' : 0,\
             'Close Short' : 0,'Close_position_price':0,'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(signals.iloc[i]['Open.S'],2),\
              'PNL' : 0,'Latent PNL':0,'Duration ':0}],ignore_index=True)

            print('\n Le ',signals.iloc[i]['Date'],', achat de ',x,' actions à',round(signals.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',round(pactol,2))
            print('Le Latent PNL est de : 0')
            TRADES.append((round(signals.iloc[i]['Open.S'],2),signals.iloc[i]['Date'],1))
            continue

        elif pos_long == 1 and signals.iloc[i]['Green_Close'] == 0 :

            trades = signals.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,'Close Long' : 0, 'Open Short' : 0,\
            'Close Short' : 0,'Close_position_price':0,'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(signals.iloc[i]['Open.S'],2), 'PNL' : 0,\
            'Latent PNL':round((signals.iloc[i]['Price'] - price_buy)*x,2),'Duration ':0}],ignore_index=True)
            try:
                MAXDD.append(round((signals.iloc[i]['Open.S'] - price_buy)*x,2))
            except:
                print('Null val of maxdd')
                MAXDD.append(0)
            print('\n Le ',str(signals.iloc[i]['Date']))
            print('Nouveau pactol : ',round(pactol,2))
            print('Le Latent PNL est de : ',MAXDD[-1])
            continue

        elif pos_long == 1 and signals.iloc[i]['Green_Close'] == 1 :
            old_pactol = round(pactol,2)
            EQ_CURX.append(signals.iloc[i]['Date'])
            EQ_CURY.append(old_pactol)
            pos_long = 0
            price_sell = round(signals.iloc[i]['Open.S'],2)
            mini_pnl = round(x*(price_sell-price_buy),2)
            TRADES.append((round(signals.iloc[i]['Open.S'],2),signals.iloc[i]['Date'],2))

            trades = trades.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,'Close Long' : 1, 'Open Short' : 0,\
            'Close Short' : 0,'Close_position_price':price_sell,'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(signals.iloc[i]['Open.S'],2),\
            'PNL' : mini_pnl,'Latent PNL':0,'Duration ':str(pd.to_timedelta(pd.to_datetime(signals.iloc[i]['Date'],format='%Y-%m-%d') - \
            pd.to_datetime(time_enter,format='%Y-%m-%d'),unit='d'))[:-8]}])

            pactol = round((pactol + mini_pnl),2)
            POS_DURATION.append(str(pd.to_timedelta(pd.to_datetime(signals.iloc[i]['Date'],format='%Y-%m-%d') - \
            pd.to_datetime(time_enter,format='%Y-%m-%d'),unit='d'))[:-8])
            print('\n Le ',str(signals.iloc[i]['Date']),', vente pour close de ',x,' actions à',round(signals.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)
            print('pnl:', mini_pnl)
            print('prix de vente',price_sell)
            print('prix d achat',price_buy)
            print('Le Latent PNL est de : 0')
            print('Duration : ',POS_DURATION[-1])
            

            if mini_pnl > 0:
                WINNERS.append(mini_pnl)
                mini_pnl = 0
                nb_win +=1
            else :
                LOOSERS.append(mini_pnl)
                mini_pnl = 0
                nb_los +=1
            continue


        elif pos_short == 0 and pos_long == 0 and signals.iloc[i]['Red']==1:
            pos_short = 1
            nbS+=1
            x = round((mise)/signals.iloc[i]['Open.S'])
            price_sell = round(signals.iloc[i]['Open.S'],2)
            time_enter = signals.iloc[i]['Date']

            trades = trades.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,'Close Long' : 0, 'Open Short' : 1,\
            'Close Short' : 0,'Close_position_price':0,'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(signals.iloc[i]['Open.S'],2), 'PNL' : 0,\
            'Latent PNL':0,'Duration ':0}],ignore_index=True)

            print('\n Le ',str(signals.iloc[i]['Date']),', vente de ',x,' actions à',round(signals.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)
            print('Le Latent PNL est de : 0')
            
            TRADES.append((round(signals.iloc[i]['Open.S'],2),signals.iloc[i]['Date'],-1))
            continue


        elif pos_short == 1 and signals.iloc[i]['Red_Close'] == 0 :

            trades = trades.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,'Close Long' : 0, 'Open Short' : 0,\
            'Close Short' : 0,'Close_position_price':0,'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(signals.iloc[i]['Open.S'],2),\
            'PNL' : 0,'Latent PNL':-round((signals.iloc[i]['Price'] - price_sell)*x,2),'Duration ':0}],ignore_index=True)

            try:
                MAXDD.append(-round((signals.iloc[i]['Open.S'] - price_sell)*x,2))
            except:
                print('Null val of maxdd')
                MAXDD.append(0)
                print('\n Le ',str(signals.iloc[i]['Date']))
                print('Nouveau pactol : ',round(pactol,2))
                print('Le Latent PNL est de : ',MAXDD[-1])
            continue

        elif pos_short == 1 and signals.iloc[i]['Red_Close'] == 1 :
            old_pactol = round(pactol,2)
            EQ_CURX.append(signals.iloc[i]['Date'])
            EQ_CURY.append(old_pactol)
            pos_short = 0
            price_buy = round(signals.iloc[i]['Open.S'],2)
            TRADES.append((round(signals.iloc[i]['Open.S'],2),signals.iloc[i]['Date'],-2))
            mini_pnl = round(x*(price_sell-price_buy),2)

            trades = trades.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,'Close Long' : 0, 'Open Short' : 0,\
            'Close Short' : 1,'Close_position_price':price_buy,'Buy/Sell Price' : price_sell,'Quantity' : x,'Actual' : round(signals.iloc[i]['Open.S'],2),\
            'PNL' : mini_pnl,'Latent PNL':0,'Duration ':str(pd.to_timedelta(pd.to_datetime(signals.iloc[i]['Date'],format='%Y-%m-%d') - \
            pd.to_datetime(time_enter,format='%Y-%m-%d'),unit='d'))[:-8]}])

            pactol = round((pactol + mini_pnl),2)
            print('\n Le ',str(signals.iloc[i]['Date']),', achat pour close de ',x,' actions à',round(signals.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)

            print('prix de vente',price_sell)
            print('prix d achat',price_buy)

            print('pnl:', mini_pnl)
            POS_DURATION.append(str(pd.to_timedelta(pd.to_datetime(signals.iloc[i]['Date'],format='%Y-%m-%d') - \
            pd.to_datetime(time_enter,format='%Y-%m-%d'),unit='d'))[:-8])
            
            print('Le Latent PNL est de : 0')
            print('Duration : ',POS_DURATION[-1])
            
            if mini_pnl > 0:
                WINNERS.append(mini_pnl)
                mini_pnl = 0
                nb_win +=1
            else :
                LOOSERS.append(mini_pnl)
                mini_pnl = 0
                nb_los +=1
            continue
            
        else:

            trades = trades.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,'Close Long' : 0, 'Open Short' : 0,\
            'Close Short' : 0,'Close_position_price':0,'Buy/Sell Price' : 0, 'Quantity' : 0,'Actual' : round(signals.iloc[i]['Open.S'],2), 'PNL' : 0,\
            'Latent PNL':0,'Duration ':0}],ignore_index=True)

    if pos_long == 1:
        pos_long = 0
        print('Cutting unresolved position')
        print(' Le price buy était de : ',price_buy)
        x = round((mise/signals.iloc[i]['Open.S']))
        print('x = ',x)
        price_sell = round(signals.iloc[i]['Open.S'],2)
        print('Et le price sell : ',price_sell)
        mini_pnl = round(x*(price_sell-price_buy),2)

        trades = trades.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,'Close Long' : 1, 'Open Short' : 0,\
        'Close Short' : 0,'Close_position_price':price_sell,'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(signals.iloc[i]['Open.S'],2),\
        'PNL' : mini_pnl,'Latent PNL':0,'Duration ':str(pd.to_timedelta(pd.to_datetime(signals.iloc[i]['Date'],format='%Y-%m-%d') - \
        pd.to_datetime(time_enter,format='%Y-%m-%d'),unit='d'))[:-8]}])

        pactol = round((pactol + mini_pnl),2)
        print('pnl : ',mini_pnl)
        if mini_pnl > 0:
            WINNERS.append(mini_pnl)
            mini_pnl = 0
            nb_win +=1
        else :
            LOOSERS.append(mini_pnl)
            mini_pnl = 0
            nb_los +=1


    if pos_short == 1:
        pos_short = 0
        print('Cutting uresolved position')
        print(' Le price sell était de : ',price_sell)
        x = round((mise)/signals.iloc[i]['Open.S'])
        print('x = ',x)
        price_buy = round(signals.iloc[i]['Open.S'],2)
        print('Et le price_buy : ',price_buy)
        mini_pnl = round(x*(price_sell-price_buy),2)

        trades = trades.append([{ 'Date': signals.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,\
        'Close_position_price':price_buy,'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(signals.iloc[i]['Open.S'],2), 'PNL' : mini_pnl,\
        'Latent PNL':0,'Duration ':str(pd.to_timedelta(pd.to_datetime(signals.iloc[i]['Date'],format='%Y-%m-%d') - \
        pd.to_datetime(time_enter,format='%Y-%m-%d'),unit='d'))[:-8]}])

        pactol = round((pactol + mini_pnl),2)
        print('pnl : ',mini_pnl)

        if mini_pnl > 0:
            WINNERS.append(mini_pnl)
            mini_pnl = 0
            nb_win +=1
        else :
            LOOSERS.append(mini_pnl)
            mini_pnl = 0
            nb_los +=1


    pnl = round(pactol - (1000000),2)
    print('\n Begining of BackTest :',signals.iloc[0]['Date'])
    print('Instrument :',ticker)
    print('Face value per trade : $',mise)
    print('End of BackTest :',signals.iloc[-1]['Date'])
    duration = str(pd.to_timedelta(pd.to_datetime(signals.iloc[-1]['Date'],format='%Y-%m-%d')\
     - pd.to_datetime(signals.iloc[0]['Date'],format='%Y-%m-%d'),unit='d'))[:-8]
    print("BackTest's period :",duration)
    print('Pnl :',round(pnl,2),'$')
    print('% Pnl : ',round(pnl/(mise)*100,2))
    if pnl == 0:
        pass
    print('Total winners :',round(sum(WINNERS),2),'$')
    print('Total loosers :',round(sum(LOOSERS),2),'$')
    print('Number of winners :',round(nb_win))
    print('Number of loosers :',round(nb_los))
    if nb_win == 0:
        nb_win += 0.00001
    if nb_los == 0:
        nb_los += 0.00001
    print('Maximum Drawdown : ',round(min(MAXDD),2))
    print('% Max Drawdown : ',round(min(MAXDD)/(mise)*100,2),'%')
    print('Nombre toatl de trades : ',round(nbL+nbS,2))
    print('% de trades Long : ',round((nbL*100)/(nbL+nbS),2))
    print('% de trades Short : ',round((nbS*100)/(nbL+nbS),2))
    try:
        print('% winners :', round((nb_win/(nb_win+nb_los) * 100),2),'%')
        print('Average winners :',(round(sum(WINNERS)/(nb_win))))
    except:
        print('% winners : null')
    try:
        print('Average losers :',(round(sum(LOOSERS)/(nb_los))))
    except:
        print('Average losers : Inf')
    try:
        for t in range(0,len(POS_DURATION)):
            average_duration = average_duration + pos_duration[t]
        average_duration = str(average_duration/(nb_los+nb_win))[:-7]
        print('Average Duration :', average_duration)
    except:
        print('No Average Duration available')
    try:
        print('Profit Factor : ',abs(round((sum(WINNERS)/sum(LOOSERS)),2)))
    except:
        print('Profit Factor : null')
    


    ########################
    ##### PLOT TRADES ######
    ########################

    #plt.figure(figsize=(26,8),dpi=300)
    #plt.title('Deep Learning '+ticker+' Trades')
    #plt.plot(signals['Open.S'],color='blue',lw=0.9)
    #for i in range(0,len(trades)):
    #    if trades[i][2] == 1:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='green',marker='^',lw=2.5)
    #    elif trades[i][2] == -1:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='red',marker='^',lw=2.5)
    #    elif trades[i][2] == 2:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='g',marker='x',lw=2.5)
    #    elif trades[i][2] == -2:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='r',marker='x',lw=2.5)

    #plt.scatter(x=signals.iloc[i]['Date'],y=signals.iloc[i]['Open.S'],c='g',marker='^',label='Open Long')
    #plt.scatter(x=signals.iloc[i]['Date'],y=signals.iloc[i]['Open.S'],c='r',marker='^',label='Open Short')
    #plt.scatter(x=signals.iloc[i]['Date'],y=signals.iloc[i]['Open.S'],c='g',marker='x',label='Close Long')
    #plt.scatter(x=signals.iloc[i]['Date'],y=signals.iloc[i]['Open.S'],c='r',marker='x',label='Close Short')
    
    #plt.legend()
    #plt.savefig('backtest de '+str(ticker)+'_02.pdf')
    #plt.close()

    #########################
    ##### PLOT E-CURVE ######
    #########################
    plt.figure(figsize=(26,8),dpi=300)
    plt.title('Deep Learning '+ticker+' Equity Curve')
    plt.plot(EQ_CURX,EQ_CURY)
    plt.scatter(x=EQ_CURX,y=EQ_CURY,c='orange',marker='o',lw=2.5)
    plt.savefig('BackTest/BT/backtest de '+str(ticker)+'_e_curve.pdf')
    plt.close()

    
    
    #print('\r Boucle ', loop, 'terminée...',end='')
    #print('Opération restantes :', len(constituents)-loop,'/',len(constituents),'\n')

    name = 'Name'
    sector = 'Sector'
    backtest = backtest.append([{'Symb.':ticker,'Name':name,'Sector':sector,'Period':average_duration,'Begin.':signals.iloc[0]['Date'],\
    'End':signals.iloc[-1]['Date'],'Pnl $':pnl,'Pnl %':round(pnl/(mise)*100,2),'Gross Profit':round(sum(WINNERS),2),'Gross Loss':round(sum(LOOSERS),2),\
    '# Winners':nb_win,'# Loosers':nb_los,'% Winners':round((nb_win/(nb_win+nb_los) * 100),2),'MaxDD $':round(min(MAXDD),2),\
    'MaxDD %':round(min(MAXDD)/(mise)*100,2),'Aver. Win $':(round(sum(WINNERS)/(nb_win+0.0001))),'Aver. Loos $':(round(sum(LOOSERS)/(nb_los+0.0001))),\
    'Aver. Duration':average_duration,'Total Trades':nbL+nbS,'% Long':round((nbL*100)/(nbL+nbS),2),'% Short':round((nbS*100)/(nbL+nbS),2),\
    'Profit Factor':abs(round((sum(WINNERS)/sum(LOOSERS)),2))}], ignore_index=True)


    #signals = signals.append([{ 'Date': list_signals[0], 'Ticker' : list_signals[8], 'Open Long' : list_signals[1],\
                               #'Close Long' : list_signals[2], 'Open Short' : list_signals[3], 'Close Short' : list_signals[4],\
                               #'Buy/Sell Price' : list_signals[5], 'Quantity' : list_signals[6], ' tPNL' : list_signals[7]}])

    #signals['Date'] = list_signals[0]
    #signals['Ticker'] = list_signals[8]
    #signals['Open Long'] = list_signals[1]
    #signals['Close Long'] = list_signals[2]
    #signals['Open Short'] = list_signals[3]
    #signals['Close Short'] = list_signals[4]
    #signals['Buy/Sell Price'] = list_signals[5]
    #signals['Quantity'] = list_signals[6]
    #signals['tPNL'] = list_signals[7]

    # symbol / name / sector / period of backtest / debut / fin / pnl / total winners / total loosers / Nb winners / nb loosers / 
    # / % winners / Max DD / Aver Win / Aver Loos / Average duration / Profit Factor
f.close()

sys.stdout = orig_stdout
tmps22=round(time.time()-tmps55,2)
time.sleep(1)
print(Fore.BLUE,"\n Time for completing the backtest = %f" %tmps22,'seconds \n',Style.RESET_ALL)
print('Erreurs de type 3 : ',len(ERROR_3))

SIMULATION DU TRADING


ValueError: min() arg is an empty sequence

In [10]:
MAXDD

[]

In [ ]:
LaBase.sort_index()

In [ ]:
BasePrep